In [1]:
import scipy.interpolate as spint
import scipy.spatial.qhull as qhull
import numpy as np

def interp_weights(xy, uv, d=2):
    tri = qhull.Delaunay(xy)
    simplex = tri.find_simplex(uv)
    vertices = np.take(tri.simplices, simplex, axis=0)
    temp = np.take(tri.transform, simplex, axis=0)
    delta = uv - temp[:, d]
    bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
    return vertices, np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))

def interpolate(values, vtx, wts):
    return np.einsum('nj,nj->n', np.take(values, vtx), wts)
    
def produce_land_sea_bool(lsm_cube):
    """    Return the land_bool and sea_bool from a given lsm_cube.
    Note: this method is not for mask; but for true value to be reserved
    """
    # Need to be true for the sea points
    sea_bool = (lsm_cube.data == 0)
    # The land points
    land_bool = (lsm_cube.data != 0)
    return land_bool, sea_bool

In [10]:
import numpy as np

# The sample data is taken from a place includes both water and sea
# (coastline at somewhere in Sri Lanka)
# For APS2, the index points lsm_aps2.data[411:415, 231:234], shape=(4, 3)
lsm_tgt_data = np.array(
    [[1, 0, 0],
     [1, 1, 0],
     [1, 1, 0],
     [1, 1, 0]])

In [9]:
# For APS3, the index points lsm_aps3.data[821:829, 461:467], shape=(8, 6)
lsm_src_data = np.array(
      [[1, 1, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 0]])

In [8]:
cube_data = np.array(
     [[300.0625, 299.1875, 300.859375, 300.4375, 300.640625, 300.8125],
      [300.734375, 303.546875, 300.828125, 300.53125, 300.578125, 300.796875],
      [302.0625, 301.078125, 300.703125, 299.90625, 300.640625, 300.71875],
      [300.796875, 299.890625, 299.703125, 300.046875, 300.71875, 300.65625],
      [297.078125, 298.875, 299.09375, 300.078125, 300.71875, 300.65625],
      [296.125, 297.875, 298.5625, 299.328125, 300.59375, 300.734375],
      [294.953125, 296.90625, 298.125, 298.828125, 298.671875, 300.765625],
      [296.578125, 297.375, 297.171875, 298.34375, 297.65625, 300.75]])

In [6]:
lat_tgt = np.array([6.328125, 6.5625, 6.796875, 7.03125])
lon_tgt = np.array([81.211053, 81.562616, 81.914179])

In [7]:
lat_src = np.array([6.26953125, 6.38671875, 6.50390625, 6.62109375,
                         6.73828125, 6.85546875, 6.97265625, 7.08984375])
lon_src = np.array([81.12304688, 81.29882812, 81.47460938,
                          81.65039062, 81.82617188, 82.00195312])

In [11]:
cube_src_sea_data = np.where((lsm_src_data==0), cube_data, np.nan)
cube_src_land_data = np.where((lsm_src_data!=0), cube_data, np.nan)

In [12]:
print(cube_src_land_data)

[[ 300.0625    299.1875           nan         nan         nan         nan]
 [ 300.734375  303.546875  300.828125         nan         nan         nan]
 [ 302.0625    301.078125  300.703125  299.90625          nan         nan]
 [ 300.796875  299.890625  299.703125  300.046875         nan         nan]
 [ 297.078125  298.875     299.09375   300.078125         nan         nan]
 [ 296.125     297.875     298.5625    299.328125         nan         nan]
 [ 294.953125  296.90625   298.125     298.828125  298.671875         nan]
 [ 296.578125  297.375     297.171875  298.34375   297.65625          nan]]


In [13]:
print(cube_src_sea_data)

[[        nan         nan  300.859375  300.4375    300.640625  300.8125  ]
 [        nan         nan         nan  300.53125   300.578125  300.796875]
 [        nan         nan         nan         nan  300.640625  300.71875 ]
 [        nan         nan         nan         nan  300.71875   300.65625 ]
 [        nan         nan         nan         nan  300.71875   300.65625 ]
 [        nan         nan         nan         nan  300.59375   300.734375]
 [        nan         nan         nan         nan         nan  300.765625]
 [        nan         nan         nan         nan         nan  300.75    ]]


In [14]:
xv, yv = np.meshgrid(lon_tgt, lat_tgt)
grid_tgt = np.dstack((yv, xv))

In [15]:
print(grid_tgt)

[[[  6.328125  81.211053]
  [  6.328125  81.562616]
  [  6.328125  81.914179]]

 [[  6.5625    81.211053]
  [  6.5625    81.562616]
  [  6.5625    81.914179]]

 [[  6.796875  81.211053]
  [  6.796875  81.562616]
  [  6.796875  81.914179]]

 [[  7.03125   81.211053]
  [  7.03125   81.562616]
  [  7.03125   81.914179]]]


In [16]:
xv, yv = np.meshgrid(lon_src, lat_src)
grid_src = np.dstack((yv, xv))

In [21]:
bool_land_src = (lsm_src_data!=0)
bool_sea_src = (lsm_src_data==0)

In [22]:
print(bool_sea_src)

[[False False  True  True  True  True]
 [False False False  True  True  True]
 [False False False False  True  True]
 [False False False False  True  True]
 [False False False False  True  True]
 [False False False False  True  True]
 [False False False False False  True]
 [False False False False False  True]]


In [23]:
bool_land_tgt = (lsm_tgt_data!=0)
bool_sea_tgt = (lsm_tgt_data==0)

In [24]:
print(bool_sea_tgt)

[[False  True  True]
 [False False  True]
 [False False  True]
 [False False  True]]


In [25]:
# xy is the source grid
xy = grid_src[bool_sea_src]
# uv is the target grid
uv = grid_tgt[bool_sea_tgt]

In [26]:
xy.shape

(17, 2)

In [27]:
uv.shape

(5, 2)

In [28]:
vtx, wts = interp_weights(xy, uv)

In [29]:
# 'values' is the coastline points to be interpolate
values = cube_src_sea_data

In [30]:
out_value_sea = interpolate(values, vtx, wts)

In [31]:
print(out_value_sea)

[ nan  nan  nan  nan  nan]


In [32]:
# xy is the source grid
xy = grid_src[bool_land_src]
# uv is the target grid
uv = grid_tgt[bool_land_tgt]

vtx, wts = interp_weights(xy, uv)

values = cube_src_land_data

out_value_land = interpolate(values, vtx, wts)

In [33]:
print(out_value_land)

[          nan           nan  302.80289337           nan           nan
           nan           nan]


In [34]:
def _closest_points_index(point, grid):
    """
    Produce the bool index for node (lat, lon) on source grid nodes
    :param point: a target grid point [lat, lon] not on the source grid
    :param grid: an input source grid
    :return: the bool value (index) on those nearest points to the point
    """
    deltas = np.abs(grid - point)
    # the grid has shape (i, j, k) i.e. (8, 6, 2) for sample testing area
    dist_2 = np.einsum('ijk,ijk->ij', deltas, deltas)
    # Index be true if the distance is minimum
    index_bool = (dist_2 == np.min(dist_2))
    return index_bool

In [35]:
pnt_tgt_sea = grid_tgt[bool_sea_tgt]

In [36]:
print(pnt_tgt_sea)

[[  6.328125  81.562616]
 [  6.328125  81.914179]
 [  6.5625    81.914179]
 [  6.796875  81.914179]
 [  7.03125   81.914179]]


In [37]:
pnt_src_sea = grid_src[bool_sea_src]
print(pnt_src_sea)

[[  6.26953125  81.47460938]
 [  6.26953125  81.65039062]
 [  6.26953125  81.82617188]
 [  6.26953125  82.00195312]
 [  6.38671875  81.65039062]
 [  6.38671875  81.82617188]
 [  6.38671875  82.00195312]
 [  6.50390625  81.82617188]
 [  6.50390625  82.00195312]
 [  6.62109375  81.82617188]
 [  6.62109375  82.00195312]
 [  6.73828125  81.82617188]
 [  6.73828125  82.00195312]
 [  6.85546875  81.82617188]
 [  6.85546875  82.00195312]
 [  6.97265625  82.00195312]
 [  7.08984375  82.00195312]]


In [39]:
index_bool_sea_tgt = _closest_points_index(pnt_tgt_sea, pnt_src_sea)

ValueError: operands could not be broadcast together with shapes (17,2) (5,2) 

# Some ideas:

In [ ]:
# Calculate the weight_index for land/sea coastline pnt
weight_index = interpolator.compute_interp_weights(tgrid)